In [4]:
from bokeh.plotting import output_notebook

output_notebook()

Loading BokehJS ...

In [5]:
import pandas as pd
embeddings = pd.read_csv('data/metadata_with_embeddings.csv', index_col=0, parse_dates=['Collection Date'])
embeddings.head()

data = embeddings[['Collection Date', 'coords0', 'coords1', 'coords2']].set_index('Collection Date').resample('M').mean()
data.head()

,coords0,coords1,coords2
Collection Date,,,
2000-01-31,0.528481,-3.737833,-0.502630
2000-02-29,0.075133,-3.591450,-0.312657
2000-03-31,0.322234,-3.725801,-0.207332
2000-04-30,NaN,NaN,NaN
2000-05-31,0.011306,-3.379667,-0.303968


In [6]:
from bokeh.palettes import viridis, inferno

palette = inferno(len(data))


In [9]:
from bokeh.plotting import figure, show

p = figure(webgl=True)
p.scatter(x=data['coords0'], y=data['coords1'], color=palette)

show(p)